In [1]:
import wave
import soundfile as sf
import pandas as pd
import numpy as np
from glob import glob

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [3]:
import pickle

In [9]:
import collections

In [6]:
yes_file_list = glob("data/yes*")
no_file_list = glob("data/no*")
none_file_list = glob("data/none*") + glob("data/extension*")
clap_file_list = glob("data/clap*")
category_list = [0 for _ in range(len(no_file_list))] + [1 for _ in range(len(yes_file_list))]+[2 for _ in range(len(none_file_list))] + [3 for _ in range(len(clap_file_list))]

In [7]:
file_list =  no_file_list + yes_file_list + none_file_list + clap_file_list

In [8]:
X = []
y = []
for file,category in zip(file_list,category_list):
    file_sf,_ = sf.read(file)
    file_sf = np.array(file_sf[:, 0])
    X.append(file_sf)
    y.append(category)

In [10]:
collections.Counter(y)

Counter({0: 66, 1: 30, 2: 135, 3: 35})

In [11]:
X = np.array(X)
y = np.array(y)
X_fft = np.array([np.fft.fft(x) for x in X])
X_fft = np.fft.fft(X)
X = np.array(
    [np.hstack((x.real**2 + x.imag**2, np.arctan2(x.real, x.imag))) for x in X_fft])

In [12]:
clf = RandomForestClassifier(n_estimators = 500,n_jobs = -1)
scores = cross_val_score(clf, X, y, cv=5)
print('score:{:.3f} (+/-{:.3f})'.format(scores.mean(), scores.std()*2))

score:0.779 (+/-0.314)


In [13]:
clf.fit(X,y)

RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [14]:
filename = 'model/several_model_add_none_class.sav'
pickle.dump(clf, open(filename, 'wb'))